# Pré-processamento de dados

Neste arquivo será realizado o pré-processamento de dados, bem como filtros temporais, espaciais e visualização de dados em gráficos.

## Aplicação dos filtros temporais

Para a aplicação dos filtros temporais, serão utilizadas as funções a seguir:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal

In [ ]:
def butter_bandpass(data, lowcut, highcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_lowpass(data, lowcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)


def butter_highpass(data, highcut, fs=200, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=200, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

## Visualização dos gráficos com e sem filtros

Power Spectrum Density - Densidade Espectral da Potência (PSD) para plotar no domínio da frequência.

PSD = |Wti|², onde o módulo é a parte real.

Notações:
- Xt: domínio do tempo
- Wf: domínio da frequência

In [1]:
def print_graphs(data):
    for i in range(data.shape[1]):
        plt.plot(data[:,i])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(data.shape[1]):
        # Fs = Frequency sample (taxa de amostragem): 250Hz
        plt.psd(data[:,i], Fs=250)
    plt.title('Domínio da frequência')
    plt.show()

In [ ]:
# Apresetnação dos dados sem aplicação dos filtros

data = np.load('../dataset-s6/data.npy')

rcParams['figure.figsize'] = [16., 5.]

print_graphs(data)

## Aplicação do filtro Notch

In [ ]:
data_filtred = butter_notch(data, 60, fs=250)
print_graphs(data_filtred)

In [ ]:
data_filtred = butter_highpass(data, 5, fs=250)
print_graphs(data_filtred)